In [1]:
import numpy as np
import pandas as pd
import pickle as p
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

In [2]:
dir = "../results3"

In [3]:
files = os.listdir(dir)

In [4]:
architecture = np.array([128, 10])

In [5]:
def avg_usage(rates, architecture):
    weights = architecture / np.sum(architecture)
    avg = np.dot(rates, weights)
    return avg

In [6]:
avg_fn = lambda x: avg_usage(x, architecture)

In [7]:
results = [p.load(open(os.path.join(dir, f), "rb")) for f in files]

In [12]:
"random_removal" in results[0].keys()

False

In [14]:
"random_removal" in results[1].keys()

True

In [17]:
def reduce_result(result):
    reduced = {}
    normal_keys = ["mask angle", "cross inhibit", "random_removal", "accuracy"]

    for k in normal_keys:
        if k in result.keys():
            reduced[k] = float(result[k])
        else:
            reduced[k] = 0.0
    
    avg_keys = ["matrix usage", "firing rates"]
    for k in avg_keys:
        reduced[k] = avg_fn(result[k])

    reduced["accuracy spiking"] = float(np.max(result["accuracy_spiking"]))
    
    return reduced

In [18]:
reduce_result(results[0])

{'mask angle': 0.24,
 'cross inhibit': 0.0,
 'random_removal': 0.0,
 'accuracy': 0.7942999601364136,
 'matrix usage': 0.4581362632089767,
 'firing rates': 0.26537425759035044,
 'accuracy spiking': 0.8127999901771545}

In [8]:
results

[{'mask angle': 0.24,
  'cross inhibit': 0.0,
  'accuracy': DeviceArray(0.79429996, dtype=float32),
  'matrix usage': array([0.48740512, 0.08349488], dtype=float32),
  'accuracy_spiking': DeviceArray([0.6529    , 0.80249995, 0.8128    , 0.8088    , 0.79789996,
               0.7624    , 0.6961    , 0.5857    , 0.4353    , 0.09999999,
               0.09999999], dtype=float32),
  'firing rates': array([0.27874006, 0.09429193])},
 {'mask angle': 0.0,
  'cross inhibit': 0.0,
  'random_removal': 0.1,
  'accuracy': DeviceArray(0.85539997, dtype=float32),
  'matrix usage': array([1., 1.], dtype=float32),
  'accuracy_spiking': DeviceArray([0.6095    , 0.75839996, 0.76669997, 0.76659995, 0.7737    ,
               0.7705    , 0.76839995, 0.7601    , 0.7532    , 0.1015    ,
               0.09999999], dtype=float32),
  'firing rates': array([0.8905958 , 0.84260186])},
 {'mask angle': 0.0,
  'cross inhibit': 0.06,
  'accuracy': DeviceArray(0.85539997, dtype=float32),
  'matrix usage': array([1.,

In [10]:
all_accuracy = np.array([r['accuracy'] for r in results])

In [11]:
matrix_usage = np.stack([avg_fn(r['matrix usage']) for r in results])

In [12]:
all_accuracy_spk = np.stack([r['accuracy_spiking'] for r in results])

In [13]:
firing_rates = np.stack([avg_fn(r['firing rates']) for r in results])

In [14]:
fig2 = go.Figure()
    
fig2.add_trace(go.Scatter(x=angles * np.pi, y=all_accuracy, name="Non-Spiking Accuracy"))
fig2.add_trace(go.Scatter(x=angles * np.pi, y=all_accuracy_spk[:,3], name="Spiking Accuracy"))

fig2.update_layout(title="Accuracy by Mask Angle",
                  xaxis_title="Mask Angle (Radians)",
                  yaxis_title="Test Accuracy")

In [15]:
fig3 = go.Figure()
    
fig3.add_trace(go.Scatter(x=matrix_usage, y=firing_rates))

fig3.update_layout(title="Firing Rate vs. Matrix Occupancy",
                  xaxis_title="Matrix Occupancy",
                  yaxis_title="Firing Rate")

In [16]:
fig4 = go.Figure()
    
fig4.add_trace(go.Scatter(x=matrix_usage, y=all_accuracy, name="Non-Spiking Accuracy"))
fig4.add_trace(go.Scatter(x=firing_rates, y=all_accuracy_spk[:,3], name="Spiking Accuracy"))

fig4.update_layout(title="Accuracy by Occupancy",
                  xaxis_title="Matrix Occupancy, Firing Rate",
                  yaxis_title="Test Accuracy")
